#Tercer intento#



*   Solo con los datos de Madrid de Idealista2018
*   Madrid_Sale.csv y Madrid_Polygons.rda bajados de https://github.com/paezha/idealista18/tree/e2cfe3f621e1c37f226ce956f61b7306c42d5721/data


*   archivo Madrid_Polygons.rda transformado en csv con Notebook en R Script R rda a csv.ipyn https://colab.research.google.com/drive/1I06zwKUreseL1p87dHq0TzMheUHiEmVc


*   Exploración de datos (EDA) copiado de https://github.com/enricobonii/Idealista-Project/blob/main/IdealistaProject%20v4.ipynb
*Agregar la Columna neighbourhood a df_Madrid: de la latitud y longitud de cada casa se busca el barrio al que corresponde en Madrid_Polygons.csv

* la variable que vamos intentar predecir va a ser la diferencia en % entre el precio del piso y el precio medio por barrio - usaremos solo los datos del cuarto trimestre, quitando los que la diferencia sea mayor del 100%

* se usan varios modelos, y luego se calcula el error en el precio





In [ ]:
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

# Ruta al archivo CSV en Google Drive
file_path = '/content/drive/My Drive/Idealista_2024/datos_2018/'

In [ ]:
from getpass import getpass
import os

# Solicita el token personal de GitHub
token = getpass('GitHub Token: ')

# Define la URL del repositorio y la ruta de clonación
repo_url = "https://github.com/edouardor/Idealista_2024.git"
destino = "/content/drive/MyDrive/Idealista_2024"

# Clona el repositorio en la carpeta de Google Drive
os.system(f'git clone https://{token}@github.com/edouardor/Idealista_2024.git {destino}')


In [ ]:
%cd /content/drive/MyDrive/Idealista_2024

In [ ]:
!ls -a


In [ ]:
!git config --global user.email "edouardo@hotmail.es"
!git config --global user.name "edouardor"

In [ ]:
# Mostrar la rama actual
!git branch


In [ ]:
# Crear y cambiar a una nueva rama
#!git checkout -b rama-limpia

# Agregar los archivos modificados
!git add .

# Hacer un commit
!git commit -m "Cambios realizados desde Google Colab en nueva rama - notebook limpiado"

# Subir la nueva rama al remoto
!git push -u origin rama-limpia


In [ ]:
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from math import radians, cos, sin, asin, sqrt
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import math

pd.set_option('display.max_columns', None)

## Initializing Datasets

###Solo Madrid


In [ ]:

df= pd.read_csv(file_path+'gdf_Madrid_with_metro_count.csv')


In [ ]:
gdf = pd.read_csv(file_path+'Madrid_Polygons_with_mean_unitprice.csv')

*para poner luego los clusters definidos para cada barrio*

In [ ]:
df.head(3)

## Exploratory Data Analysis

In [ ]:
print(df.dtypes)

From here we can observe that the formatted variable types make sense with the expected and are homogeneous among the different datasets.

In [ ]:
df.describe(include='all')

In [ ]:
# prompt: drop observertions where df[df['ROOMNUMBER']>20]
print(f"Hay {len(df[df['ROOMNUMBER']>20])} pisos con mas de 20 habitaciones")
print("los quitamos")

df = df[df['ROOMNUMBER'] <= 20]

In [ ]:
# dime cuales son los valores medios de UNITPRICE para cada periodo, y la diferencia en % con el primer periodo - con dos decimales es suficiente - pon tambien cuantos datos hay en df para cada periodo


# Group data by 'PERIOD' and calculate the mean of 'UNITPRICE'
mean_unitprice_by_period = df.groupby('PERIOD')['UNITPRICE'].agg(['mean', 'count'])

# Calculate the percentage difference from the first period
first_period_mean = mean_unitprice_by_period['mean'].iloc[0]
mean_unitprice_by_period['pct_change'] = ((mean_unitprice_by_period['mean'] - first_period_mean) / first_period_mean) * 100

# Format the output
mean_unitprice_by_period['pct_change'] = mean_unitprice_by_period['pct_change'].round(2)
mean_unitprice_by_period = mean_unitprice_by_period.rename(columns={'mean':'mean_unitprice', 'count':'n_datapoints'})

mean_unitprice_by_period

In [ ]:
df.loc[:, 'Precio_medio_m2'] = df.groupby(['neighbourhood', 'PERIOD'])['UNITPRICE'].transform('mean')


In [ ]:
df.head(3)

###El valor a predecir va a ser la diferencia en % entre el precio por m2 del piso y el precio por m2 del barrio en el que se encuentra

In [ ]:
# prompt: pon una columna con (UNITPRICE - Precio_medio_m2)/Precio_medio_m2

# Calculate the new column
df['diferencia_precio_m2'] = (df['UNITPRICE'] - df['Precio_medio_m2']) / df['Precio_medio_m2']

In [ ]:
# prompt: haz un histograma con diferencia_precio_m2

# Create the histogram
plt.figure(figsize=(10, 6))
sns.histplot(df['diferencia_precio_m2'], kde=True)
plt.title('Histograma de la diferencia de precio por m2')
plt.xlabel('Diferencia de precio por m2')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
# prompt: dime cuantos valores hay superiores a 1

# Assuming 'df' is your DataFrame and it has a column named 'diferencia_precio_m2'
count_greater_than_1 = len(df[df['diferencia_precio_m2'] > 1])
print(f"Hay {count_greater_than_1} valores superiores a 1 en la columna 'diferencia_precio_m2'.")
df = df[df['diferencia_precio_m2'] <= 1]
print("los quitamos")


In [ ]:
# Calcular los percentiles para el 1% inferior y superior
lower_bound = df['PRICE'].quantile(0.01)  # 1% inferior
upper_bound = df['PRICE'].quantile(0.99)  # 1% superior

formatted_lower_bound = "{:,.0f}".format(lower_bound)
formatted_upper_bound = "{:,.0f}".format(upper_bound)

print(f"Vamos a quitar los pisos con un precio inferior a {formatted_lower_bound} euros")
print(f"y los que tengan un precio superior a {formatted_upper_bound} euros")

pisos_inicial = len(df)
print(f"Hay {pisos_inicial} pisos en total")

# Filtrar el DataFrame para conservar solo el 99% de los datos
df= df[(df['PRICE'] >= lower_bound) & (df['PRICE'] <= upper_bound)]

# Calculate the number of rows removed
removed_rows = pisos_inicial - len(df)

print(f"Hemos quitado {removed_rows} pisos")

print(f"Quedan {len(df)} pisos")

El primer predictor posible del precio es el precio medio del m2 por barrio y periodo múltiplicado por los m2.

Vamos a calcular cuanto sería el error medio en valor absoluto que cometemos así, y esto nos servirá como escenario base que mejorar.

In [ ]:
# prompt: cual es el error (Mape) entre PRICE y CONSTRUCTEDAREA * Precio_medio_m2 ?

# Calculate the predicted price based on CONSTRUCTEDAREA and Precio_medio_m2
df['predicted_price'] = df['CONSTRUCTEDAREA'] * df['Precio_medio_m2']

# Calculate the Mean Absolute Percentage Error (MAPE)
df['percentage_error'] = abs((df['PRICE'] - df['predicted_price']) / df['PRICE']) * 100

mape = df['percentage_error'].mean()

print(f"El MAPE entre PRICE y CONSTRUCTEDAREA * Precio_medio_m2 es: {mape:.2f}%")

## Multivariate Analysis

Vamos a agrupar los diferentes barrios en clusters, de barrios 'baratos' a barrios caros.

Para evitar problemas, vamos a usar solo los datos del cuarto trimestre.


In [ ]:
df4 = df[df['PERIOD'] == 4]

**Determinamos el Número Óptimo de Grupos**

*Antes de realizar la agrupación, es útil determinar cuántos grupos (clusters) son apropiados. Una técnica común para esto es el método del codo.*

In [ ]:
from sklearn.cluster import KMeans

# Lista para almacenar la inercia
inercia = []
K = range(1, 10)

for k in K:
    modelo = KMeans(n_clusters=k, random_state=42)
    modelo.fit(df4[['Precio_medio_m2']])
    inercia.append(modelo.inertia_)

# Graficar el método del codo
plt.figure(figsize=(8, 5))
plt.plot(K, inercia, 'bo-')
plt.xlabel('Número de clusters (k)')
plt.ylabel('Inercia')
plt.title('Método del Codo para Determinar el Número Óptimo de Clusters')
plt.show()


*Buscamos un "codo" en la gráfica donde la inercia comienza a disminuir de manera menos pronunciada. Aquí k=6*

**Aplicar K-Means para Agrupar los Barrios**

In [ ]:
# Definir el número de clusters
k_optimo = 6

# Inicializar y ajustar el modelo
kmeans = KMeans(n_clusters=k_optimo, random_state=42)
df['Cluster'] = kmeans.fit_predict(df[['Precio_medio_m2']])

# Opcional: Añadir los centroides
centroides = kmeans.cluster_centers_


In [ ]:
# Ordenar los barrios por precio medio y cluster
df.sort_values('Precio_medio_m2', inplace=True)


# Visualización con Seaborn
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='neighbourhood', y='Precio_medio_m2', hue='Cluster', palette='viridis')
plt.xticks(rotation=45)
plt.xlabel('Barrio')
plt.ylabel('Precio Medio m² (€)')
plt.title('Agrupación de Barrios por Precio Medio del Metro Cuadrado')
plt.legend(title='Cluster')
plt.tight_layout()
plt.show()


**Nos Aseguramos la Consistencia de los Nombres de Barrios entre los ficheros**

In [ ]:
df['Cluster'].value_counts()

In [ ]:
# Convertir a mayúsculas y eliminar espacios adicionales para asegurar coincidencias
df_Madrid['neighbourhood'] = df_Madrid['neighbourhood'].str.upper().str.strip()
gdf['LOCATIONNAME'] = gdf['LOCATIONNAME'].str.upper().str.strip()

# Opcional: Verificar barrios que no coinciden
barrios_df = set(df_Madrid['neighbourhood'])
barrios_gdf = set(gdf['LOCATIONNAME'])
barrios_faltantes = barrios_df - barrios_gdf
if barrios_faltantes:
    print("Barrios en precio_medio_barrio que no están en gdf:", barrios_faltantes)

barrios_faltantes_gdf = barrios_gdf - barrios_df
if barrios_faltantes_gdf:
    print("Barrios en gdf que no están en precio_medio_barrio:", barrios_faltantes_gdf)


In [ ]:
gdf.head()

In [ ]:
import geopandas as gpd
from shapely import wkt

# Asegúrate de que 'WKT_geometry' está en formato string
gdf['WKT_geometry'] = gdf['WKT_geometry'].astype(str)

# Convertir la columna WKT a objetos geométricos
gdf['geometry'] = gdf['WKT_geometry'].apply(wkt.loads)


# Asegurarse de que 'gdf' es un GeoDataFrame
# Si no lo es, convertirlo
gdf = gpd.GeoDataFrame(gdf, geometry='geometry')  # Ajusta según corresponda

# Fusionar los DataFrames
gdf = gdf.merge(df_Madrid[['neighbourhood', 'Cluster']], how='left',
                left_on='LOCATIONNAME', right_on='neighbourhood')

# Opcional: Verificar si hay valores NaN en 'Cluster' después de la fusión
barrios_sin_cluster = gdf[gdf['Cluster'].isna()]['LOCATIONNAME'].unique()
if len(barrios_sin_cluster) > 0:
    print("Barrios sin asignar a un cluster:", barrios_sin_cluster)

# Eliminar la columna 'Barrio' si ya no es necesaria
gdf.drop(columns=['neighbourhood'], inplace=True)


In [ ]:
# Supongamos que tu GeoDataFrame se llama 'gdf' y tiene una columna 'Cluster'
# y una columna 'mean_UNITPRICE' que representa el precio medio por barrio.

# Calcular el precio medio por cluster
precio_medio_por_cluster = gdf.groupby('Cluster')['mean_UNITPRICE'].mean().reset_index()

print(precio_medio_por_cluster)


In [ ]:
# Ordenar los clusters por 'mean_UNITPRICE' de menor a mayor
precio_medio_por_cluster = precio_medio_por_cluster.sort_values('mean_UNITPRICE').reset_index(drop=True)

print(precio_medio_por_cluster)


In [ ]:
# Crear una lista de clusters ordenados por precio medio
clusters_ordenados = precio_medio_por_cluster['Cluster'].tolist()

# Crear un diccionario de mapeo: {cluster_original: cluster_nuevo}
mapeo_clusters = {cluster_original: nuevo_cluster for nuevo_cluster, cluster_original in enumerate(clusters_ordenados)}

print(mapeo_clusters)


In [ ]:
# Aplicar el mapeo para crear una nueva columna 'Cluster_Ordenado'
gdf['Cluster_Ordenado'] = gdf['Cluster'].map(mapeo_clusters)

# Opcional: Si deseas reemplazar la columna 'Cluster' original
# gdf_geo['Cluster'] = gdf_geo['Cluster'].map(mapeo_clusters)


In [ ]:
# Definir una paleta de colores
palette = sns.color_palette('coolwarm', k_optimo).as_hex()

# Asignar colores a cada cluster
gdf['color'] = gdf['Cluster_Ordenado'].apply(lambda x: palette[x])

# Plotear el mapa
fig, ax = plt.subplots(figsize=(10, 10))
gdf.plot(color=gdf['color'], edgecolor='black', ax=ax)

# Añadir una leyenda
for cluster in range(k_optimo):
    ax.scatter([], [], c=palette[cluster], label=f'Cluster {cluster}')
ax.legend(title='Cluster')

plt.title('Agrupación de Barrios por Precio Medio del Metro Cuadrado')
plt.show()


In [ ]:
gdf.head()

**Seleccionamos las Columnas Relevantes de gdf**


In [ ]:
# Seleccionar columnas necesarias
clusters_info = gdf[['LOCATIONNAME', 'Cluster_Ordenado']]


In [ ]:
clusters_info

In [ ]:
# Convertir a mayúsculas y eliminar espacios al inicio y al final
df_Madrid['neighbourhood'] = df_Madrid['neighbourhood'].str.upper().str.strip()
gdf['LOCATIONNAME'] = gdf['LOCATIONNAME'].str.upper().str.strip()


**Realizamos la Fusión (Merge) de df_Madrid con gdf**

In [ ]:
# Fusionar df_Madrid con clusters_info
df_Madrid = df_Madrid.merge(clusters_info, how='left', left_on='neighbourhood', right_on='LOCATIONNAME')

# Opcional: Eliminar la columna 'LOCATIONNAME' si ya no es necesaria
df_Madrid.drop(columns=['LOCATIONNAME'], inplace=True)


In [ ]:
df_Madrid.head()

**Verificamos la Asignación de Clusters**

In [ ]:
# Verificar si hay pisos sin asignar a un cluster
pisos_sin_cluster = df[df['Cluster'].isna()]
num_sin_cluster = pisos_sin_cluster.shape[0]

if num_sin_cluster > 0:
    print(f"Hay {num_sin_cluster} pisos sin asignar a un cluster:")
    print(pisos_sin_cluster[['neighbourhood']].drop_duplicates())
else:
    print("Todos los pisos han sido asignados a un cluster.")


In [ ]:
sns.boxplot(data=df, x='Cluster', y='UNITPRICE')

plt.title('Distribución de Precios por Cluster de Barrio')
plt.xlabel('Cluster Ordenado')
plt.ylabel('Precio Medio m² (€)')
plt.show()


## ML Models

In [ ]:
df.columns

In [ ]:
# prompt: crea una nueva columna llamada BUILTTYPE que valga 1 si BUITTYPEID_1 vale 1, 2 si BUITTYPEID_2 vale 1 y 3 si BUITTYPEID_3 vale 1

# Create the 'BUILTTYPE' column based on the conditions
df['BUILTTYPE'] = 0  # Initialize the column with 0
df.loc[df['BUILTTYPEID_1'] == 1, 'BUILTTYPE'] = 1
df.loc[df['BUILTTYPEID_2'] == 1, 'BUILTTYPE'] = 2
df.loc[df['BUILTTYPEID_3'] == 1, 'BUILTTYPE'] = 3

In [ ]:

madrid = df.drop(columns=['ASSETID','PRICE', 'UNITPRICE', 'ADTYPOLOGYID',
       'ADOPERATIONID','AMENITYID','HASNORTHORIENTATION', 'HASSOUTHORIENTATION',
       'HASEASTORIENTATION', 'HASWESTORIENTATION','FLATLOCATIONID',
       'CADMAXBUILDINGFLOOR', 'CADDWELLINGCOUNT', 'CADASTRALQUALITYID','DISTANCE_TO_METRO',
       'LONGITUDE', 'LATITUDE', 'CITYNAME', 'ADTYPOLOGY', 'ADOPERATION',
       'neighbourhood', 'metros_within_500m','BUILTTYPEID_1','BUILTTYPEID_2','BUILTTYPEID_3','DISTANCE_TO_CITY_CENTER', 'DISTANCE_TO_STREET','predicted_price', 'percentage_error'])

In [ ]:
madrid.columns

In [ ]:
madrid = madrid[madrid['PERIOD']==4]

In [ ]:
madrid.drop(columns=['PERIOD'], inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error
import statsmodels.api as sm

X = madrid.drop(['diferencia_precio_m2'], axis=1)
y = madrid['diferencia_precio_m2']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
y_test.shape

In [ ]:
X_train_sm = sm.add_constant(X_train)
X_test_sm = sm.add_constant(X_test)

model_sm = sm.OLS(y_train, X_train_sm).fit()

y_pred = model_sm.predict(X_test_sm)

mae = mean_absolute_error(y_test, y_pred)
mdae = median_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print(f'MAE: {mae}')
print(f'MDAE: {mdae}')
print(f'RMSE: {rmse}')
print(f'MAPE: {mape}%')

print(model_sm.summary())

In [ ]:
# prompt: define una función con X_test y y_pred como input, y MAE, MSE, Medae y mape como output

import numpy as np
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error

def calculate_metrics(X_test, y_pred, y_test):
    """
    Calculates MAE, MSE, MedAE, and MAPE.

    Args:
        X_test: Test features (not used in the calculations, but included for consistency with the original code).
        y_pred: Predicted values.
        y_test: True values.

    Returns:
        A dictionary containing MAE, MSE, MedAE, and MAPE.
    """
    indices_test = X_test.index
    price_test = df.loc[indices_test, 'UNITPRICE'] * df.loc[indices_test, 'CONSTRUCTEDAREA']

    unitprice_pred = (1 + y_pred)  * X_test['Precio_medio_m2']
    price_pred = unitprice_pred * X_test['CONSTRUCTEDAREA']

    mae = mean_absolute_error(price_test, price_pred)
    mse = mean_squared_error(price_test, price_pred)
    medae = median_absolute_error(price_test, price_pred)
    mape = np.mean(np.abs((price_test- price_pred) / price_test)) * 100

    return {'mae': mae, 'mse': mse, 'medae': medae, 'mape': mape}

In [ ]:

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# 3. Entrenar el modelo
modelo = LinearRegression()
modelo.fit(X_train, y_train)

# 4. Realizar predicciones en el conjunto de prueba
y_pred = modelo.predict(X_test)


# imprimir error medio en precios, en %
error_medio_precios = calculate_metrics(X_test, y_pred, y_test)['mape']
print(f'Error medio en precios con regresión lineal: {error_medio_precios:.2f}%')


In [ ]:
import shap

# Crear un objeto explainer
explainer = shap.Explainer(modelo, X_train)
shap_values = explainer(X_test)

# Visualizar los valores SHAP
shap.summary_plot(shap_values, X_test)


*Best parameters found: {'bootstrap': True, 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}*

In [ ]:

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
#from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error
#import matplotlib.pyplot as plt


rf = RandomForestRegressor(random_state=42)

param_grid = {
    'n_estimators': [300], #[100, 200, 300]
    'max_depth': [None], #[10, 20, None]
    'min_samples_split': [2], #[2, 5, 10]
    'min_samples_leaf': [1], #[1, 2, 4]
    'bootstrap': [True] #[True, False]
}

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

grid_search.fit(X_train, y_train)

best_rf = grid_search.best_estimator_

y_pred = best_rf.predict(X_test)


print(f'Best parameters found: {grid_search.best_params_}')

feature_importances = best_rf.feature_importances_
features = X.columns

importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print(importance_df)

plt.figure(figsize=(10, 6))
plt.barh(importance_df['Feature'], importance_df['Importance'])
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Feature Importance in Random Forest Model')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# imprimir error medio en precios, en %
error_medio_precios = calculate_metrics(X_test, y_pred, y_test)['mape']
print(f'Error medio en precios con random forest: {error_medio_precios:.2f}%')

*Best parameters found: {'colsample_bytree': 0.6, 'learning_rate': 0.05, 'max_depth': 10, 'n_estimators': 300, 'reg_alpha': 0.1, 'reg_lambda': 0, 'subsample': 1.0}*


In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb = XGBRegressor(random_state=42)

param_grid = {
    'n_estimators': [300], #[100, 200, 300],
    'max_depth': [10], #[3, 5, 7, 10],
    'learning_rate':  [0.01, 0.05, 0.1, 0.2], #[0.05],
    'subsample': [1], #[0.6, 0.8, 1.0],
    'colsample_bytree':[0.6, 0.8, 1.0],
    'reg_alpha': [0, 0.1, 0.5, 1],
    'reg_lambda': [0, 0.1, 0.5, 1]
}

grid_search = GridSearchCV(estimator=xgb, n_jobs=-1, param_grid=param_grid, cv=5, verbose=10, scoring='neg_mean_squared_error')

grid_search.fit(X_train, y_train)

best_xgb = grid_search.best_estimator_

y_pred = best_xgb.predict(X_test)

print(f'Best parameters found: {grid_search.best_params_}')

Fitting 5 folds for each of 6912 candidates, totalling 34560 fits


In [ ]:
# imprimir error medio en precios, en %
error_medio_precios = calculate_metrics(X_test, y_pred, y_test)['mape']
print(f'Error medio en precios con xgboost: {error_medio_precios:.2f}%')

Error medio en precios con xgboost: 13.08%


In [ ]:
# Solo si deseas verificar (no recomendado para evaluar rendimiento)
y_pred_all = best_xgb.predict(X)
error_medio_precios = calculate_metrics(X, y_pred_all, y)['mape']
print(f'Error medio en precios totales con xgboost: {error_medio_precios:.2f}%')

Error medio en precios totales con xgboost: 8.64%
